In [1]:
from openai import AssistantEventHandler
from typing_extensions import override
from openai import OpenAI

In [2]:
client = OpenAI(api_key="")

In [4]:

assistant = client.beta.assistants.create(
    name="Math Tutor 2",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o-mini",
)


In [5]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print([f"assitant {a.id} and name: {a.name}" for a in my_assistants.data])

['assitant asst_OKK3dLzKgwZDAIlmJSeHLPGt and name: Math Tutor 2', 'assitant asst_FHelaEujPggsGvVkuoYOHHo1 and name: Math Tutor 2', 'assitant asst_1PJdlUxPNPwrE6TolbaCWjdg and name: None', 'assitant asst_XRhVjpwkQMZTH3WBf5MqetYy and name: None', 'assitant asst_TYhg1Zypz2bLNGFsps54bSAP and name: None', 'assitant asst_31xdWePXKw2sGgit8XSgmWhg and name: None', 'assitant asst_nf84xtSxetR0R2qLj7BcR7R5 and name: None', 'assitant asst_IsWLrOozqZ9X2WPgz12Bj26M and name: None', 'assitant asst_Qf3dXdCsaykPw7xjZhDLCE6B and name: None', 'assitant asst_HA6Hodb6T9vI9iGVPoozaq6A and name: None', 'assitant asst_triOIFx8hOSIEYPur7nSt9bP and name: None', 'assitant asst_7nxIe1yV64IekfACuvLEvkBl and name: None', 'assitant asst_iBU0WrYAIkNMiOkZ7cXIZqnD and name: None', 'assitant asst_A4JIs705zGOZJhrDinP50B1I and name: None', 'assitant asst_e4Y9c5EZCrEXtfUrG4ClMDEG and name: None', 'assitant asst_n1DRLwPsgKiXUdDGrHQNa5Mq and name: None', 'assitant asst_JEBOctmi9z1oVIN8ifpoGMoz and name: None', 'assitant asst

In [6]:

thread = client.beta.threads.create()

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)


# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

# Then, we use the `stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.


with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()




assistant > code_interpreter

import sympy as sp

# Define the variable
x = sp.symbols('x')

# Define the equation
equation = sp.Eq(3*x + 11, 14)

# Solve the equation
solution = sp.solve(equation, x)
solution
assistant > The solution to the equation \( 3x + 11 = 14 \) is \( x = 1 \).

In [8]:
class StreamEventHandler(AssistantEventHandler):
    # Iterator over just the text deltas in the stream
    @override
    def on_text_created(self, text) -> None:
        # Callback that is fired when a text content block is created
        # print(f"\nassistant > ", end="", flush=True)
        pass

    @override
    def on_text_delta(self, delta, snapshot):
        """Callback that is fired whenever a text content delta is returned
        by the API.

        The first argument is just the delta as sent by the API and the second argument
        is the accumulated snapshot of the text. For example:

        on_text_delta(TextDelta(value="The"), Text(value="The")),
        on_text_delta(TextDelta(value=" solution"), Text(value="The solution")),
        on_text_delta(TextDelta(value=" to"), Text(value="The solution to")),
        on_text_delta(TextDelta(value=" the"), Text(value="The solution to the")),
        on_text_delta(TextDelta(value=" equation"), Text(value="The solution to the equivalent")),
        """
        # print(delta.value, end="", flush=True)
        pass

In [14]:
stream = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  stream=True
)

for event in stream:
  # if event.data.status == 'completed':
  print(event)
  if event.event != "thread.message.delta":
    continue

  # for content_delta in event.data.delta.content or []:
  #   if content_delta.type == "text" and content_delta.text and content_delta.text.value:
  #       print(content_delta.text.value)
  print(event.data.delta.content[0].text.value)


ThreadRunCreated(data=Run(id='run_DhEPZ3aVjsQzxaOUD3pm2YWd', assistant_id='asst_OKK3dLzKgwZDAIlmJSeHLPGt', cancelled_at=None, completed_at=None, created_at=1722475127, expires_at=1722475727, failed_at=None, incomplete_details=None, instructions='You are a personal math tutor. Write and run code to answer math questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_ZndzViFqCouqyN7MH0GS2FQB', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={'code_interpreter': {'file_ids': []}}), event='thread.run.created')
ThreadRunQueued(data=Run(id='run_DhEPZ3aVjsQzxaOUD3pm2YWd', assistant_id='asst_OKK3dLzKgwZDAIlmJSeHLPGt', cancelled_at=None, compl

In [25]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
) as stream:
    if stream.on_text_created:
        print('@@@@ start the big bang')
    if stream.on_text_delta:
        # for text in stream.text_deltas:
        # print(*stream.text_deltas, end="", flush=True)
        print('-------')
        print(stream.)
    if stream.on_end:
        print('####syaonara')

SyntaxError: invalid syntax (1711408863.py, line 11)

In [ ]:
assistant = client.beta.assistants.retrieve(
                "session.assistant_id")

NotFoundError: Error code: 404 - {'error': {'message': "No assistant found with id 'session.assistant_id'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [15]:
stream = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  stream=True
)

for event in stream:
  # Print each event for debugging purposes
  print(event)

  # Check for when the run is completed
  if event.event == "thread.run.completed":
    # Access and print usage stats
    usage = event.data.usage
    print("Run Completed:")
    print(f"Prompt Tokens: {usage.prompt_tokens}")
    print(f"Completion Tokens: {usage.completion_tokens}")
    print(f"Total Tokens: {usage.total_tokens}")
    break  # Exit the loop after capturing the final stats

  # Continue processing other events as needed
  if event.event == "thread.message.delta":
    # Handle message deltas
    print(event.data.delta.content[0].text.value)


ThreadRunCreated(data=Run(id='run_dihrhxSndbtjzGhQB5bzSJAw', assistant_id='asst_OKK3dLzKgwZDAIlmJSeHLPGt', cancelled_at=None, completed_at=None, created_at=1722475180, expires_at=1722475780, failed_at=None, incomplete_details=None, instructions='You are a personal math tutor. Write and run code to answer math questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_ZndzViFqCouqyN7MH0GS2FQB', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={'code_interpreter': {'file_ids': []}}), event='thread.run.created')
ThreadRunQueued(data=Run(id='run_dihrhxSndbtjzGhQB5bzSJAw', assistant_id='asst_OKK3dLzKgwZDAIlmJSeHLPGt', cancelled_at=None, compl